# Pituitary Referrals Data Analytics Notebook

This notebook contains some scripts for generating tables and insights based off the pituitary referrals data Dr. Catalino shared with me.

In [52]:
import os
from pathlib import Path

import pandas as pd
import numpy as np

## Load & Clean Data

### Loading Data

In [53]:
raw_ref_data = pd.read_csv(os.path.join("data", "ref_data.csv"), index_col=0, encoding="ISO-8859-1")

In [54]:
raw_ref_data.head()

,Zip Code,Date of Initial Contact,Records Requested,Records Received,Films Requested,Films Received,Date of IV,Date of Surgery,RP Degree,RP Specialty,RP Address
Date of Birth,,,,,,,,,,,
17-Feb-93,33613,08-Oct-03,08-Oct-03,10-Oct-03,08-Oct-03,10-Oct-03,14-Nov-03,NaN,MD,NSR,"880 6th Street South, Suite 450 St Petersburg..."
02-Apr-70,22942,30-Oct-14,30-Oct-14,30-Oct-14,30-Oct-14,31-Oct-14,26-Nov-14,NaN,MD,PCP,NaN
05-Mar-87,22401,01-Jul-03,01-Jul-03,NaN,01-Jul-03,NaN,NaN,NaN,MD,GYN,"621 Jefferson Davis Highway, Suite 201 Freder..."
23-Jan-57,24472,13-Jan-14,13-Jan-14,NaN,13-Jan-14,NaN,NaN,NaN,MD,END,"55 Comfort Way, Suite 1 Lexington, VA 24450"
01-Dec-60,24504,14-Jul-15,NaN,14-Jul-15,NaN,13-Aug-15,NaN,NaN,MD,NSG,"2025 Tate Springs Road Lynchburg ,VA 24501"


### Clean Data

#### Start with Degree

There are 27 (after all being set to lowercase) unique names for degrees. Let's make a map for this.

In [55]:
referrers_degree = raw_ref_data["RP Degree"].apply(lambda x: str(x).lower()).unique()
len(referrers_degree), referrers_degree

(27,
 array(['md', 'nan', 'md, phd', 'do', 'np', 'fnp', 'pa-c', 'pa', 'fmd',
        'rn', 'aprn', 'm. d.', 'jr', 'md,facs', 'm.d.', 'phd',
        'dsc md frs', 'rnc', 'md\nmd', 'ph.d.', 'od', 'cnm', 'md, ph.d',
        'nc', 'md ph.d', 'n.p.', 'jr.'], dtype=object))

In [56]:
referrers_degree_map = {
    "md": "md",
    "md, phd": "md",
    "fmd": "md",
    "m. d.": "md",
    "md,facs": "md",
    "m.d.": "md",
    "md\\nmd": "md",
    "md, ph.d.": "md",
    "md ph.d": "md",
    "dsc md frs": "md",
    "do": "do",
    "od": "do",
    "nan": "nan",
    "np": "np",
    "fnp": "np",
    "n.p.": "np",
    "pa-c": "pa",
    "pa": "pa",
    "rn": "rn",
    "aprn": "rn",
    "rnc": "rn",
    "nc": "rn",
    "cnm": "rn",
    "jr": "jr",
    "jr.": "jr",
}

# Now use the mapping and apply it to the column's lowercase values
referrers_degree = raw_ref_data["RP Degree"].apply(lambda x: str(x).lower()).map(referrers_degree_map)
# Turn nans into "nan"
referrers_degree = referrers_degree.fillna("nan")
# Check the unique values after mapping
referrers_degree.unique()


array(['md', 'nan', 'do', 'np', 'pa', 'rn', 'jr'], dtype=object)

In [57]:
# Now we can replace the original column with the cleaned one
raw_ref_data["RP Degree"] = referrers_degree

#### Now Let's Clean Specialty

There is wayyyyy more of these.

In [58]:
# Start with seeing all the possible values of RP Specialty and RP Degree
referrers_spec = raw_ref_data["RP Specialty"].apply(lambda x: str(x).lower()).unique()
len(referrers_spec), referrers_spec

(193,
 array(['nsr', 'pcp', 'gyn', 'end', 'nsg', 'nan', 'neu', 'endo',
        'employer', 'dds', 'radiology-oncology', 'ref.', 'ref',
        'self/ref.', 'neuro surg', 'obs', 'int', 'opt', 'family physician',
        'onc', 'uro', 'fam med', 'ob', 'neurosurgeon', 'endocrinologist',
        'internist', 'general surgery', 'otolaryngologist', 'sur', 'peds',
        'endo.', 'ort', 'fmd', 'rad', 'ent', 'ped', 'attending', 'eye',
        'end/ref', 'ob/gyn', 'er', 'neuro/ref', 'distant relative/nsr',
        'family practice', 'psy', 'ob gyn', 'primary care', 'arthritis',
        'do', 'ref/pcp', "dau's pcp", 'neurologist', 'neuro-surg.', 'nsy',
        'neurology', 'endocrinology', 'surgeon', 'pcp/ref.', 'rad onc',
        'pul', 'opth', 'ped endo', 'genetics', 'neurosurgery',
        'neuro-surg,ref.,friend', 'internal medicine', 'nih', 'allergy',
        'family medicine', 'ref/endo', 'np', 'pain mgt', 'immunology',
        'endo/ref', 'cards', 'rheu', 'ophth', 'family medince', 'md',

In [59]:
specialty_to_category = {
    "Primary Care": [
        'pcp', 'primary care', 'family physician', 'fam med', 'family medicine',
        'fmd', 'family practice', 'fam medicine', 'fam practice', 'fp', 'gp',
        'general medicine', 'internal medicine', 'internal med', 'int',
        'int med', 'internist', 'im', 'student health', 'pcp at facility',
        'int.', 'internal med.', 'general'
    ],
    "Endocrinology": [
        'end', 'endo', 'endocrinologist', 'endocrinology', 'endocronolgist',
        'endocinolgist', 'endocrinolgist', 'endo/internal medicine',
        'internal medicine/endo', 'endo/ref', 'ref/endo', 'endo/ref.', 'end./ref.',
        'ref./endo.', 'endo./ref.', 'end fellow'
    ],
    "Neurology": [
        'neu', 'neurologist', 'neurology', 'neurologists', 'neurgologist',
        'neuro', 'neuro/ref', 'neuro-onco', 'neur'
    ],
    "Neurosurgery": [
        'nsg', 'nsy', 'neuro surg', 'neurosurgeon', 'neurosurgery', 
        'neuro-surg.', 'neuro-surg,ref.,friend', 'neurosurg/ref', 'neurosurg.', 
        'nsgy', 'mj neurosciences', 'nero-surg', 'neuro-sergon'
    ],
    "Surgery": [
        'sur', 'surgeon', 'general surgery', 'oral surgery'
    ],
    "OB/GYN": [
        'gyn', 'ob', 'ob gyn', 'obgyn', 'ob/gyn', 'ob/gyn & ref',
        'ob/ gyn', 'ob/ref', 'gyn/pcp', 'gynecology'
    ],
    "Oncology": [
        'onc', 'hem onc', 'oncology', 'rad onc', 'radiology-oncology',
        'roc', 'rad oncology', 'rad on'
    ],
    "Radiology": [
        'rad', 'rad on', 'rad onc', 'radiology-oncology'
    ],
    "Orthopedics": [
        'ort', 'ortho', 'orth', 'sports & spine'
    ],
    "Ophthalmology": [
        'opt', 'ophth', 'opth', 'optho', 'opthalmology', 'opthamologist'
    ],
    "Pediatrics": [
        'peds', 'ped', 'pediatrics', 'ped endo', 'peds endo', 'ped/end',
        'peds card', 'ped/pcp', 'ped end'
    ],
    "Urology": [
        'uro', 'uro/ref', 'urology', 'uro./ref'
    ],
    "Cardiology": [
        'car', 'card', 'cardiac', 'cardiologist', 'car/pcp'
    ],
    "ENT": [
        'ent', 'oto', 'ent/ref', 'otolaryngologist', 'oto-hns', 'ot/ref.'
    ],
    "Rheumatology": [
        'rheu', 'rhe', 'rhem'
    ],
    "Psychiatry": [
        'psy', 'psych'
    ],
    "Pulmonology": [
        'pul'
    ],
    "Nephrology": [
        'neph', 'nephrology', 'nep'
    ],
    "Allergy/Immunology": [
        'allergy', 'immunology'
    ],
    "Pain Management": [
        'pain', 'pain mgt'
    ],
    "Hospital Medicine": [
        'attending', 'hospitalist', 'hosp'
    ],
    "Emergency Medicine": [
        'er', 'er physician', 'emd', 'ems'
    ],
    "Genetics": [
        'genetics'
    ],
    "Infectious Disease": [
        'id'
    ],
    "GI / Digestive Health": [
        'gi/dig health'
    ],
    "Rehabilitation": [
        'pmr'
    ],
    "Endoscopy": [
        'end/ref'
    ],
    "Other / Unclassified": [
        'nsr', 'nan', 'self/ref.', 'ref.', 'ref', 'ref/pcp', 'ref./pcp',
        'pcp/ref.', 'pcp/ref', 'ref/end', 'ref/int', 'ref/int./pcp',
        'int/ref', 'old pcp/ref', 'ref/', 'ref./endo.', "dau's pcp",
        'distant relative/nsr', 'midlife', 'andrology', 'ems',
        'ems', 'student health', 'inter', 'epid', 'fellow', 'md', 'nrs',
        'oco', 'ref./', 'nrs', 'mj neurosciences', 'pts niece', 'end/ref.'
    ]
}


# Create a mapping dictionary for the specialties to their categories
specialty_to_category_map = {}
for category, specialties in specialty_to_category.items():
    for specialty in specialties:
        specialty_to_category_map[specialty] = category
# Now use the mapping and apply it to the column's lowercase values
referrers_spec = raw_ref_data["RP Specialty"].apply(lambda x: str(x).lower()).map(specialty_to_category_map)
# Turn nans into "nan"
referrers_spec = referrers_spec.fillna("nan")
# Check the unique values after mapping
referrers_spec.unique()

array(['Other / Unclassified', 'Primary Care', 'OB/GYN', 'Endocrinology',
       'Neurosurgery', 'Neurology', 'nan', 'Radiology', 'Ophthalmology',
       'Oncology', 'Urology', 'Surgery', 'ENT', 'Pediatrics',
       'Orthopedics', 'Hospital Medicine', 'Endoscopy',
       'Emergency Medicine', 'Psychiatry', 'Pulmonology', 'Genetics',
       'Allergy/Immunology', 'Pain Management', 'Rheumatology',
       'Nephrology', 'Rehabilitation', 'Cardiology', 'Infectious Disease',
       'GI / Digestive Health'], dtype=object)

In [60]:
# Now we can replace the original column with the cleaned one
raw_ref_data["RP Specialty"] = referrers_spec

In [61]:
raw_ref_data.head

<bound method NDFrame.head of               Zip Code Date of Initial Contact Records Requested  \
Date of Birth                                                      
17-Feb-93        33613               08-Oct-03         08-Oct-03   
02-Apr-70        22942               30-Oct-14         30-Oct-14   
05-Mar-87        22401               01-Jul-03         01-Jul-03   
23-Jan-57        24472               13-Jan-14         13-Jan-14   
01-Dec-60        24504               14-Jul-15               NaN   
...                ...                     ...               ...   
31-Jul-69        23188               12-Oct-15               NaN   
12-Jul-47        24445               21-Mar-12         21-Mar-12   
12-Jan-46        22202               25-May-06         25-May-06   
12-Jan-46        22513                     NaN               NaN   
24-Mar-98        22101               28-Jan-10         28-Jan-10   

              Records Received Films Requested Films Received Date of IV  \
Date of B

#### Do the same for zip code 

In [62]:
# See the unique values of Zip Code
zips = raw_ref_data["Zip Code"].unique()

# See that all the zips are 5 digits long, if not try to resolve this
def clean_zip(zip):
    """
    Clean the zip code to ensure it's a 5-digit number.

    Args:
        zip (str or int): The zip code to clean.

    Returns:
        str: The cleaned zip code. 'nan' if the zip code is not valid.
    """
    if len(str(zip)) != 5:
        # If it contains a dash, split it and take the first part
        if "-" in str(zip):
            zip = str(zip).split("-")[0]
            # Check if it's 5 digits now
            if len(zip) != 5:
                print(f"Zip code {zip} is not 5 digits long after splitting.")
                return "nan"
        else:
            print(f"Zip code {zip} is not 5 digits long.")
            return "nan"
        
        # If it's np.nan, return 'nan'
    if pd.isna(zip):
        return "nan"
    return str(zip)

# Apply the function to the Zip Code column
raw_ref_data["Zip Code"] = raw_ref_data["Zip Code"].apply(clean_zip)

Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code ISRAEL is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code H3W 1P4 is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code V6G 3G4 is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code nan is not 5 digits long.
Zip code 

### Do Some Quick Analytics

In [63]:
# Function to get the difference in time between two dates
def get_time_difference(start_date, end_date):
    """
    Calculate the difference in days between two dates.
    Handles NaN values by returning NaN if either date is NaN. Otherwise if a date is not in the correct format, it returns NaN.

    Args:
        start_date (str): The start date in 'YYYY-MM-DD' format.
        end_date (str): The end date in 'YYYY-MM-DD' format.

    Returns:
        int: The difference in days between the two dates.
    """
    
    # Check if either date is NaN
    if pd.isna(start_date) or pd.isna(end_date):
        return np.nan
    
    # Check if the dates are in the correct format
    if not isinstance(start_date, str) or not isinstance(end_date, str):
        return np.nan
    
    # Check if the dates are in the correct format of 08-Oct-03	
    try:
        start_date = pd.to_datetime(start_date, format='%d-%b-%y')
        end_date = pd.to_datetime(end_date, format='%d-%b-%y')
    except ValueError:
        return np.nan
    
    # Convert to datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # Calculate the difference
    time_diff = end_date - start_date
    
    # Return the difference in days
    return time_diff.days

# Apply the function to the DataFrame, get the time between Records Requested and Records Received, Films Requested and Films Received, Date of Initial Contact and Date of IV, and Date of Initial Contact and Date of Surgery
raw_ref_data["Days between Records Requested and Received"] = raw_ref_data.apply(
    lambda row: get_time_difference(row["Records Requested"], row["Records Received"]), axis=1
)
raw_ref_data["Days between Films Requested and Received"] = raw_ref_data.apply(
    lambda row: get_time_difference(row["Films Requested"], row["Films Received"]), axis=1
)
raw_ref_data["Days between Initial Contact and IV"] = raw_ref_data.apply(
    lambda row: get_time_difference(row["Date of Initial Contact"], row["Date of IV"]), axis=1
)
raw_ref_data["Days between Initial Contact and Surgery"] = raw_ref_data.apply(
    lambda row: get_time_difference(row["Date of Initial Contact"], row["Date of Surgery"]), axis=1
)


In [64]:
raw_ref_data.head()

,Zip Code,Date of Initial Contact,Records Requested,Records Received,Films Requested,Films Received,Date of IV,Date of Surgery,RP Degree,RP Specialty,RP Address,Days between Records Requested and Received,Days between Films Requested and Received,Days between Initial Contact and IV,Days between Initial Contact and Surgery
Date of Birth,,,,,,,,,,,,,,,
17-Feb-93,33613,08-Oct-03,08-Oct-03,10-Oct-03,08-Oct-03,10-Oct-03,14-Nov-03,NaN,md,Other / Unclassified,"880 6th Street South, Suite 450 St Petersburg...",2.0,2.0,37.0,NaN
02-Apr-70,22942,30-Oct-14,30-Oct-14,30-Oct-14,30-Oct-14,31-Oct-14,26-Nov-14,NaN,md,Primary Care,NaN,0.0,1.0,27.0,NaN
05-Mar-87,22401,01-Jul-03,01-Jul-03,NaN,01-Jul-03,NaN,NaN,NaN,md,OB/GYN,"621 Jefferson Davis Highway, Suite 201 Freder...",NaN,NaN,NaN,NaN
23-Jan-57,24472,13-Jan-14,13-Jan-14,NaN,13-Jan-14,NaN,NaN,NaN,md,Endocrinology,"55 Comfort Way, Suite 1 Lexington, VA 24450",NaN,NaN,NaN,NaN
01-Dec-60,24504,14-Jul-15,NaN,14-Jul-15,NaN,13-Aug-15,NaN,NaN,md,Neurosurgery,"2025 Tate Springs Road Lynchburg ,VA 24501",NaN,NaN,NaN,NaN


In [68]:
# Get some stats for each referrer degree type
referrers_degree_stats = raw_ref_data.groupby("RP Degree").agg({
    "Days between Records Requested and Received": ["mean", "std"],
    "Days between Films Requested and Received": ["mean", "std"],
    "Days between Initial Contact and IV": ["mean", "std"],
    "Days between Initial Contact and Surgery": ["mean", "std"]
}).reset_index()

# Show the results in nice tables
referrers_degree_stats.columns = [' '.join(col).strip() for col in referrers_degree_stats.columns.values]
referrers_degree_stats.columns = ["RP Degree", "Days between Records Requested and Received Mean",
                                   "Days between Records Requested and Received Std",
                                   "Days between Films Requested and Received Mean",
                                   "Days between Films Requested and Received Std",
                                   "Days between Initial Contact and IV Mean",
                                   "Days between Initial Contact and IV Std",
                                   "Days between Initial Contact and Surgery Mean",
                                   "Days between Initial Contact and Surgery Std"]
referrers_degree_stats = referrers_degree_stats.round(2)
referrers_degree_stats = referrers_degree_stats.rename(columns={"RP Degree": "Referrer Degree"})
# Add the number of records for each degree type
referrers_degree_stats["Number of Records"] = raw_ref_data.groupby("RP Degree").size().values
# Sort by the number of records in descending order
referrers_degree_stats = referrers_degree_stats.sort_values(by="Number of Records", ascending=False)
referrers_degree_stats.head(20)

,Referrer Degree,Days between Records Requested and Received Mean,Days between Records Requested and Received Std,Days between Films Requested and Received Mean,Days between Films Requested and Received Std,Days between Initial Contact and IV Mean,Days between Initial Contact and IV Std,Days between Initial Contact and Surgery Mean,Days between Initial Contact and Surgery Std,Number of Records
2,md,14.92,76.13,17.80,111.06,64.89,351.57,56.18,124.99,4278
3,nan,20.52,66.80,22.38,63.79,100.04,269.77,75.07,84.49,921
4,np,9.74,26.01,13.19,30.13,84.32,359.53,54.67,43.06,99
0,do,15.51,53.06,16.48,53.17,41.82,30.99,46.67,10.25,73
5,pa,3.10,3.31,88.54,296.17,38.20,18.14,NaN,NaN,23
6,rn,3.00,3.67,8.00,1.41,83.67,69.51,NaN,NaN,8
1,jr,1.00,NaN,5.00,NaN,NaN,NaN,NaN,NaN,2


In [67]:
# Get the same stats for each referrer specialty type
referrers_spec_stats = raw_ref_data.groupby("RP Specialty").agg({
    "Days between Records Requested and Received": ["mean", "std"],
    "Days between Films Requested and Received": ["mean", "std"],
    "Days between Initial Contact and IV": ["mean", "std"],
    "Days between Initial Contact and Surgery": ["mean", "std"]
}).reset_index()
# Show the results in nice tables
referrers_spec_stats.columns = [' '.join(col).strip() for col in referrers_spec_stats.columns.values]
referrers_spec_stats.columns = ["RP Specialty", "Days between Records Requested and Received Mean",
                                   "Days between Records Requested and Received Std",
                                   "Days between Films Requested and Received Mean",
                                   "Days between Films Requested and Received Std",
                                   "Days between Initial Contact and IV Mean",
                                   "Days between Initial Contact and IV Std",
                                   "Days between Initial Contact and Surgery Mean",
                                   "Days between Initial Contact and Surgery Std"]
referrers_spec_stats = referrers_spec_stats.round(2)
referrers_spec_stats = referrers_spec_stats.rename(columns={"RP Specialty": "Referrer Specialty"})
# Add the number of records for each specialty type
referrers_spec_stats["Number of Records"] = raw_ref_data.groupby("RP Specialty").size().values
# Sort by the number of records in descending order
referrers_spec_stats = referrers_spec_stats.sort_values(by="Number of Records", ascending=False)
referrers_spec_stats.head(20)

,Referrer Specialty,Days between Records Requested and Received Mean,Days between Records Requested and Received Std,Days between Films Requested and Received Mean,Days between Films Requested and Received Std,Days between Initial Contact and IV Mean,Days between Initial Contact and IV Std,Days between Initial Contact and Surgery Mean,Days between Initial Contact and Surgery Std,Number of Records
4,Endocrinology,15.54,50.48,18.01,128.65,80.68,430.96,56.08,46.61,1971
17,Other / Unclassified,16.74,62.22,25.38,90.03,80.35,367.80,49.48,45.34,1419
20,Primary Care,9.37,47.31,11.37,22.08,39.85,205.26,43.59,31.97,1123
11,Neurology,31.70,206.41,27.28,186.58,76.03,276.14,171.40,557.18,300
15,Ophthalmology,27.00,163.65,41.44,185.61,27.21,19.10,30.07,13.14,113
28,nan,8.19,19.03,11.65,19.83,37.77,54.37,52.00,31.00,90
13,OB/GYN,10.13,21.68,18.47,33.04,49.81,28.74,55.00,NaN,81
2,ENT,3.70,4.74,4.26,3.27,36.17,17.22,22.00,11.31,46
27,Urology,4.60,7.49,8.35,11.90,41.27,17.00,34.00,NaN,44
19,Pediatrics,27.13,85.77,10.10,11.23,59.12,89.82,30.33,14.22,43


In [69]:
# Now lets do the same by zip code
# Get the stats for each zip code
referrers_zip_stats = raw_ref_data.groupby("Zip Code").agg({
    "Days between Records Requested and Received": ["mean", "std"],
    "Days between Films Requested and Received": ["mean", "std"],
    "Days between Initial Contact and IV": ["mean", "std"],
    "Days between Initial Contact and Surgery": ["mean", "std"]
}).reset_index()
# Show the results in nice tables
referrers_zip_stats.columns = [' '.join(col).strip() for col in referrers_zip_stats.columns.values]
referrers_zip_stats.columns = ["Zip Code", "Days between Records Requested and Received Mean",
                                   "Days between Records Requested and Received Std",
                                   "Days between Films Requested and Received Mean",
                                   "Days between Films Requested and Received Std",
                                   "Days between Initial Contact and IV Mean",
                                   "Days between Initial Contact and IV Std",
                                   "Days between Initial Contact and Surgery Mean",
                                   "Days between Initial Contact and Surgery Std"]
referrers_zip_stats = referrers_zip_stats.round(2)
referrers_zip_stats = referrers_zip_stats.rename(columns={"Zip Code": "Referrer Zip Code"})
# Add the number of records for each zip code
referrers_zip_stats["Number of Records"] = raw_ref_data.groupby("Zip Code").size().values
# Sort by the number of records in descending order
referrers_zip_stats = referrers_zip_stats.sort_values(by="Number of Records", ascending=False)
referrers_zip_stats.head(20)

,Referrer Zip Code,Days between Records Requested and Received Mean,Days between Records Requested and Received Std,Days between Films Requested and Received Mean,Days between Films Requested and Received Std,Days between Initial Contact and IV Mean,Days between Initial Contact and IV Std,Days between Initial Contact and Surgery Mean,Days between Initial Contact and Surgery Std,Number of Records
1982,nan,10.60,18.21,12.39,18.17,50.26,33.94,52.75,41.81,239
524,22901,6.35,9.25,31.33,77.09,57.00,75.14,NaN,NaN,71
499,22801,2.04,2.54,2.43,3.58,26.73,76.90,NaN,NaN,51
833,24401,9.00,17.86,3.25,4.62,49.17,37.39,12.00,NaN,43
554,22980,8.44,9.63,6.73,10.34,31.75,21.32,NaN,NaN,40
526,22903,16.44,37.57,6.86,3.80,40.21,16.71,NaN,NaN,40
428,22407,5.39,5.57,8.56,8.17,39.79,18.54,55.50,19.91,38
479,22701,9.45,12.81,12.00,10.92,54.83,41.93,21.00,NaN,36
869,24502,2.82,2.74,11.56,18.80,49.12,35.39,82.00,NaN,35
228,20147,64.00,197.77,57.88,184.86,34.12,101.15,27.67,10.50,32
